In [29]:
# !pip install selenium

###Load libraries
import awscli
import selenium
import boto3
import pandas as pd
import time

from selenium import webdriver

## Part I. Create web-scraper to load csv file into S3 Bucket 

In [30]:
####
#Author: brandon chiazza
#version 1.0
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##


####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome(r"C:\Users\11946/chromedriver.exe")

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element_by_css_selector('table.Bordered')

print(table)


<selenium.webdriver.remote.webelement.WebElement (session="aec6f06d8102b179aecc25545d73feca", element="7cb212d0-5ccf-46e9-abb5-eeadf4471231")>


In [31]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])

#remove the blank row in the csv output file
df = df.iloc[1:]

display(df) #let's have a look at the data before creating the CSV file and loading it into s3 

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
2,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
3,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
4,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
5,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
6,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
7,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
8,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
9,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY
10,1000 Islands Hose Haulers,45-38-38,454570241,NFP,CARTHAGE,NY


In [16]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3:/jinming/'#specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3:/jinming/charities_bureau_scrape_20210407232059.csv


# Part II. Update web-scraper to iterate all results and load csv file into S3 Bucket

In [11]:
while True:
    current_page_number = int(driver.find_element_by_css_selector('li[data-testid=current-page-item]').text)

    print(f"Processing page {current_page_number}..")
 
        next_page_link = driver.find_element_by_xpath(f'.//li[span = "{current_page_number + 1}"]')
        next_page_link.click()
    except NoSuchElementException:
        print(f"Exiting. Last page: {current_page_number}.")
        break

   # TODO: save the page

IndentationError: unexpected indent (<ipython-input-11-1bcf8566039e>, line 6)

import requests

df1 =[]

for page in range(10):

    print('---', page, '---')

    r = requests.get(table + str(page))

    # String substitution for HTML
    for link in soup.find_all("a"):
        print("<a href='>%s'>%s</a>" % (link.get("href"), link.text))

    # Fetch and print general data from title class
    general_data = soup.find_all('div', {"Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"})

    for item in general_data:
        print(item.contents[0].text)
        print(item.contents[1].text.replace('.',''))
        print(item.contents[2].text)

In [16]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [27]:
df =[]
#url = "https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp"
#headers = {"Accept-Language": "en-US, en;q=0.5"}
results = requests.get(url, headers=headers)

soup = BeautifulSoup(results.text, "html.parser")

#initiate data storage
Organization_Name = []
NY_Reg_Number = []
EIN = []
Registrant_Type = []
City = []
State = []


df = soup.find_all('div', class_='lister-item mode-advanced')

#our loop through each container
for container in df:

  # Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State      
        #Organization_Name = container.h3.a.text
        # titles.append(Organization_Name)
        
        #year
        Organization_Name = container.h3.find('span', class_='Organization Name').text
        name.append(Organization_Name)

        # runtime
        NY_Reg_Number = container.p.find('span', class_='NY Reg #').text if container.p.find('span', class_='NY Reg #').text else '-'
        number.append(NY_Reg_Number)

        #IMDb rating
        EIN = float(container.strong.text)
        EIN.append(EIN)

        #metascore
        Registrant_Type = container.find('span', class_='Registrant Type').text if container.find('span', class_='Registrant Type') else '-'
        Registrant_Type.append(Registrant_Type)

        
        City = container.h3.find('span', class_='City').text
        city.append(City)

        
        State = container.h3.find('span', class_='State').text
        state.append(State)

#pandas dataframe        
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])

#remove the blank row in the csv output file
df = df.iloc[1:]

display(df) #let's have a look at the data before creating the CSV file and loading it into s3 

,Organization Name,NY Reg #,EIN,Registrant Type,City,State


In [15]:
import requests
from bs4 import BeautifulSoup as bs
  
URL = 'https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp'
  
req = requests.get(URL)
soup = bs(req.text, 'html.parser')
  
titles = soup.find_all('div',attrs = {"Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"})
  
print(titles)

[]


In [28]:

import requests
from bs4 import BeautifulSoup as bs

  
for page in range(1,10):
      # pls note that the total number of
    # pages in the website is more than 5000 so i'm only taking the
    # first 10 as this is just an example
  
    req = requests.get(URL + str(page) + '/')
    soup = bs(req.text, 'html.parser')
  
    titles = soup.find_all('div',attrs={'class','head'})
  
    for i in range(4,19):
        if page>1:
            print(f"{(i-3)+page*15}" + titles[i].text)
        else:
            print(f"{i-3}" + titles[i].text)

IndexError: list index out of range